### Content
This compiled dataset pulled from four other datasets linked by time and place, and was built to find signals correlated to increased suicide rates among different cohorts globally, across the socio-economic spectrum.

### References
United Nations Development Program. (2018). Human development index (HDI). Retrieved from http://hdr.undp.org/en/indicators/137506

World Bank. (2018). World development indicators: GDP (current US$) by country:1985 to 2016. Retrieved from http://databank.worldbank.org/data/source/world-development-indicators#

[Szamil]. (2017). Suicide in the Twenty-First Century [dataset]. Retrieved from https://www.kaggle.com/szamil/suicide-in-the-twenty-first-century/notebook

World Health Organization. (2018). Suicide prevention. Retrieved from http://www.who.int/mental_health/suicide-prevention/en/

### Inspiration
Suicide Prevention.

Цель лабораторной работы получить практические знания по работе с признаками на известном датасете статистики самоубийств.

Вам необходимо будет подготовить данные для обучения линейной модели предсказания количества самоубийств (столбец - suicides/100k pop).

Чек-лист:
0. Изучите файл annotation.txt. Там содержится информация о датасете.
1. Загрузите датасет data.csv.
2. Посмотрите на данные. Отобразите общую информацию по признакам (вспомните о describe и info). Напишите в markdown свои наблюдения.
3. Выявите пропуски, а также возможные причины их возникновения. Решите, что следует сделать с ними. Напишите в markdown свои наблюдения.
4. Оцените зависимости переменных между собой. Используйте корреляции. Будет хорошо, если воспользуетесь profile_report. Напишите в markdown свои наблюдения.
5. Определите стратегию преобразования категориальных признаков (т.е. как их сделать адекватными для моделей).
6. Найдите признаки, которые можно разделить на другие, или преобразовать в другой тип данных. Удалите лишние, при необходимости.
7. Разделите выборку на обучаемую и тестовую.
8. Обучите линейную модель. Напишите в markdown свои наблюдения по полученным результатам.

Если возникнут затруднения, то смотрите на материал практических занятий. Данного там должно хватить для выполнения всех пунктов. Желаю успеха!

In [329]:
import pandas as pd
import numpy as np

import pandas_profiling

import seaborn as sns
from matplotlib import pyplot as plt

**1. Загрузите датасет data.csv.**

In [136]:
data = pd.read_csv('data.csv')
data.head()

,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [9]:
#list(data)

**0. Изучите файл annotation.txt. Там содержится информация о датасете.**

- sex - пол
- age - возрастная группа
- suicides_no - количество суицидов
- population - численность населения
- suicides/100k pop - число самоубийств на 100к жителей
- country-year - страна/год
- HDI for year - Индекс человеческого развития. составной индекс, измеряющий средние достижения в трех основных измерениях человеческого развития - долгая и здоровая жизнь, знания и достойный уровень жизни.
- gdp_for_year - ВВП за год 
- gdp_per_capita - ВВП на душу начеления
- generation - поколение

**2. Посмотрите на данные. Отобразите общую информацию по признакам (вспомните о describe и info). Напишите в markdown свои наблюдения.**

In [5]:
data.describe()

,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
count,27820.000000,2.782000e+04,27820.000000,8364.000000,27820.000000
mean,242.574407,1.844794e+06,12.816097,0.776601,16866.464414
std,902.047917,3.911779e+06,18.961511,0.093367,18887.576472
min,0.000000,2.780000e+02,0.000000,0.483000,251.000000
25%,3.000000,9.749850e+04,0.920000,0.713000,3447.000000
50%,25.000000,4.301500e+05,5.990000,0.779000,9372.000000
75%,131.000000,1.486143e+06,16.620000,0.855000,24874.000000
max,22338.000000,4.380521e+07,224.970000,0.944000,126352.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27820 entries, 0 to 27819
Data columns (total 10 columns):
sex                   27820 non-null object
age                   27820 non-null object
suicides_no           27820 non-null int64
population            27820 non-null int64
suicides/100k pop     27820 non-null float64
country-year          27820 non-null object
HDI for year          8364 non-null float64
 gdp_for_year ($)     27820 non-null object
gdp_per_capita ($)    27820 non-null int64
generation            27820 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 2.1+ MB


**Вывод:** в целом всё неплохо. Есть всего один столбец с пропусками, но их около 70%. Есть выбросы по количеству суицидов. Разница между 75% квартилем и максимальным значением очень большая. Столбец 'gdp_for_year ($)' имеет числовые значения, однако в датасете они тип - object

**3. Выявите пропуски, а также возможные причины их возникновения. Решите, что следует сделать с ними. Напишите в markdown свои наблюдения.**

Именно пропуски присутствуют только в одном столбце. Возможные причины - данный показатель не считался в той или иной стране в указанном году. С одной стороны характеристика 'HDI for year' интересная и ее хотелось бы оставить, но с другой стороны очеь большое количество недостающих данных. Заполнять их медианными или модальными значениями значениями вполне возможно, размах у этого признака довольно маленький. Но если мы 70% заполним одинаковыми значениями пользы от этого показателя будет немного.  
Попробуем два варианта(если успею):
- удалить столбец 'HDI for year' целиком
- удалить все строки не содержащие значения 'HDI for year'

**4. Оцените зависимости переменных между собой. Используйте корреляции. Будет хорошо, если воспользуетесь profile_report. Напишите в markdown свои наблюдения.**

In [7]:
pandas_profiling.ProfileReport(data)

C:\Users\PC-Home\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,10
Number of observations,27820
Total Missing (%),7.0%
Total size in memory,2.1 MiB
Average record size in memory,80.0 B
Numeric,5
Categorical,5
Boolean,0
Date,0
Text (Unique),0
Rejected,0


**Вывод:** 
- столбцы 'suicides_no'(a), 'population'(b) являются составными столбцами 'suicides/100k pop'(c). Формула c=a/b * 100k. Так что если у нас целевая функция 'suicides/100k pop', то прогнозировать это значение имея, 'suicides_no', 'population' не интересно
- столбец 'age' и столбец 'generation' имеют по 6 уникальных значений, но они не взаимозаменяемы, так как в каждой из этих подгрупп разное количество элементов.  
- забавно, но датасет имеет по равному количеству элементов в столбце 'sex'.  
- выброс по показателю 'suicides_no' уже отмечал.  
- 'HDI for year' имеет достаточно высокую корреляцию с 'gdp_per_capita ($)', поэтому выберем вариант с удалением столбца 'HDI for year', так как он имеет большое количество пропусков и есть показатель, коррелирующий с ним очень сильно.


**5. Определите стратегию преобразования категориальных признаков (т.е. как их сделать адекватными для моделей).**

Будем преобразовывать пол и возраст и поколение по методу OHE. Использовать будем или возраст, или покаление.

**6. Найдите признаки, которые можно разделить на другие, или преобразовать в другой тип данных. Удалите лишние, при необходимости.**

In [143]:
# Уберем из названий столбцов пробелы
data.rename(columns={'suicides/100k pop': 'suicides/100k_pop',
                     'HDI for year': 'HDI_for_year',
                     ' gdp_for_year ($) ': 'gdp_for_year_$',
                     ' gdp per capita ($) ': 'gdp_for_capita_$'}, inplace=True)

In [146]:
# Не могу понять но последний столбец не переименовался из кодa в предыдущей строки. Повторю процедуру
data.rename(columns={'gdp_per_capita ($)': 'gdp_for_capita_$'}, inplace=True)

In [147]:
data.head()

,sex,age,suicides_no,population,suicides/100k_pop,country-year,HDI_for_year,gdp_for_year_$,gdp_for_capita_$,generation
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [151]:
# Преобразуем столбец 'gdp_for_year ($)' в числовой
data['gdp_for_year_$'] = data['gdp_for_year_$'].str.strip().str.replace(',', '').astype('int64')

In [154]:
# Разделим страну и год
data['country'] = data['country-year'].str[:-4]
data['year'] = data['country-year'].str[-4:].astype(int)

In [160]:
# Это не работает, просто оставил на память этапов поиска решения убрать запятые
#for i in len(data.iloc[:, [7]]):
#    data.iloc[i, [7]] = int(str(data.iloc[i, [7]].values).replace(',', '').strip("['']"))

In [234]:
# Исключим выбросы по 'suicides_no'. Уберем значения больше 95-th percentile - 1050 (взято из profile)
df_1 = data[data['suicides_no'] <= 1050]
df_2 = df_1.reset_index().drop(['index'], axis=1) # чтобы не было каши с индексами

In [236]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26429 entries, 0 to 26428
Data columns (total 12 columns):
sex                  26429 non-null object
age                  26429 non-null object
suicides_no          26429 non-null int64
population           26429 non-null int64
suicides/100k_pop    26429 non-null float64
country-year         26429 non-null object
HDI_for_year         8004 non-null float64
gdp_for_year_$       26429 non-null int64
gdp_for_capita_$     26429 non-null int64
generation           26429 non-null object
country              26429 non-null object
year                 26429 non-null int32
dtypes: float64(2), int32(1), int64(4), object(5)
memory usage: 2.3+ MB


In [315]:
# Разложим по OHE столбец 'sex'
from sklearn import preprocessing

In [238]:
sex = df_2['sex'].to_numpy().reshape(-1, 1) # хотя не очень понимаю зачем здесь reshape

In [247]:
oh_encoder = preprocessing.OneHotEncoder()
oh_encoder.fit(sex)
oh_result1 = oh_encoder.transform(sex).toarray()
#oh_result1

In [243]:
sex_columns_names = df_2['sex'].unique()
sex_columns = ['sex_{}'.format(i) for i in reversed(sex_columns_names)] # reversed чтобы 1 стояла там где надо
#sex_df.index = df_2.index
sex_df = pd.DataFrame(oh_result1, columns=sex_columns)
df_sexOHE = pd.concat([df_2, sex_df], axis=1)

In [246]:
#df_sexOHE.head()

In [376]:
# Разложим по OHE столбец 'age'
age = df_2['age'].to_numpy().reshape(-1, 1) # хотя не очень понимаю зачем здесь reshape

In [377]:
oh_encoder = preprocessing.OneHotEncoder()
oh_encoder.fit(age)
oh_result2 = oh_encoder.transform(age).toarray()
#oh_result2

In [378]:
a = df_sexOHE['age'].unique()

In [379]:
a[1], a[2], a[3], a[4], a[5]  = a[3], a[1], a[5], a[4], a[2] # чтобы наименования группам соответствовали

In [380]:
age_columns = ['{}'.format(i) for i in a]
age_df = pd.DataFrame(oh_result2, columns=a)
df_age_sex_OHE = pd.concat([df_sexOHE, age_df], axis=1)

In [381]:
df_age_sex_OHE.head()

,sex,age,suicides_no,population,suicides/100k_pop,country-year,HDI_for_year,gdp_for_year_$,gdp_for_capita_$,generation,country,year,sex_female,sex_male,15-24 years,25-34 years,35-54 years,5-14 years,55-74 years,75+ years
0,male,15-24 years,21,312900,6.71,Albania1987,NaN,2156624900,796,Generation X,Albania,1987,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,male,35-54 years,16,308000,5.19,Albania1987,NaN,2156624900,796,Silent,Albania,1987,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,female,15-24 years,14,289700,4.83,Albania1987,NaN,2156624900,796,Generation X,Albania,1987,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,male,75+ years,1,21800,4.59,Albania1987,NaN,2156624900,796,G.I. Generation,Albania,1987,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,male,25-34 years,9,274300,3.28,Albania1987,NaN,2156624900,796,Boomers,Albania,1987,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [382]:
df_age_sex_OHE.drop(['HDI_for_year', 'sex', 'age', 'country-year'], axis=1, inplace=True)

In [383]:
# Отбор столбцов для первой модели

In [413]:
X = df_age_sex_OHE.drop(['generation', 'suicides/100k_pop', 'country', 'gdp_for_year_$', 'population'], axis=1)
y = df_age_sex_OHE['suicides/100k_pop']

In [414]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [415]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

estimator = LinearRegression()
estimator.fit(X_train, y_train) # Обучение


y_pred = estimator.predict(X_test)

print("R2: \t", r2_score(y_test, y_pred))
print("RMSE: \t", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: \t", mean_absolute_error(y_test, y_pred))

R2: 	 0.3229841155779729
RMSE: 	 14.757881311670177
MAE: 	 9.299343171874549


In [387]:
# Разложим по OHE столбец 'generation'
generation = df_age_sex_OHE['generation'].to_numpy().reshape(-1, 1) # хотя не очень понимаю зачем здесь reshape

In [388]:
oh_encoder = preprocessing.OneHotEncoder()
oh_encoder.fit(generation)
oh_result3 = oh_encoder.transform(generation).toarray()
#oh_result3

In [389]:
g = df_age_sex_OHE['generation'].unique()

In [390]:
gen_columns = ['{}'.format(i) for i in g]
gen_df = pd.DataFrame(oh_result3, columns=g)
df_age_sex_gen_OHE = pd.concat([df_age_sex_OHE, gen_df], axis=1)

In [391]:
df_age_sex_gen_OHE.head()

,suicides_no,population,suicides/100k_pop,gdp_for_year_$,gdp_for_capita_$,generation,country,year,sex_female,sex_male,...,35-54 years,5-14 years,55-74 years,75+ years,Generation X,Silent,G.I. Generation,Boomers,Millenials,Generation Z
0,21,312900,6.71,2156624900,796,Generation X,Albania,1987,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,16,308000,5.19,2156624900,796,Silent,Albania,1987,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,14,289700,4.83,2156624900,796,Generation X,Albania,1987,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,21800,4.59,2156624900,796,G.I. Generation,Albania,1987,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,9,274300,3.28,2156624900,796,Boomers,Albania,1987,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [422]:
X = df_age_sex_gen_OHE.drop(['generation', 'suicides/100k_pop', 'country', 'year', 'population'], axis=1)
y = df_age_sex_gen_OHE['suicides/100k_pop']

In [423]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [424]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

estimator = LinearRegression()
estimator.fit(X_train, y_train) # Обучение


y_pred = estimator.predict(X_test)

print("R2: \t", r2_score(y_test, y_pred))
print("RMSE: \t", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: \t", mean_absolute_error(y_test, y_pred))

R2: 	 0.3352204191677225
RMSE: 	 14.623907107897486
MAE: 	 9.133980735491225


In [425]:
pandas_profiling.ProfileReport(df_age_sex_gen_OHE)

C:\Users\PC-Home\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,22
Number of observations,26429
Total Missing (%),0.0%
Total size in memory,4.3 MiB
Average record size in memory,172.0 B
Numeric,6
Categorical,2
Boolean,14
Date,0
Text (Unique),0
Rejected,0


In [426]:
# Это, конечно, "проклятье размерности", но разложим по OHE столбец 'country'
country = df_age_sex_gen_OHE['country'].to_numpy().reshape(-1, 1) # хотя не очень понимаю зачем здесь reshape

In [427]:
oh_encoder = preprocessing.OneHotEncoder()
oh_encoder.fit(country)
oh_result4 = oh_encoder.transform(country).toarray()
#oh_result4

In [428]:
coun = df_age_sex_gen_OHE['country'].unique()

In [429]:
coun_columns = ['{}'.format(i) for i in coun]
coun_df = pd.DataFrame(oh_result4, columns=coun)
df_age_sex_gen_coun_OHE = pd.concat([df_age_sex_gen_OHE, coun_df], axis=1)

In [430]:
df_age_sex_gen_coun_OHE.head()

,suicides_no,population,suicides/100k_pop,gdp_for_year_$,gdp_for_capita_$,generation,country,year,sex_female,sex_male,...,Thailand,Trinidad and Tobago,Turkey,Turkmenistan,Ukraine,United Arab Emirates,United Kingdom,United States,Uruguay,Uzbekistan
0,21,312900,6.71,2156624900,796,Generation X,Albania,1987,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,16,308000,5.19,2156624900,796,Silent,Albania,1987,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,14,289700,4.83,2156624900,796,Generation X,Albania,1987,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,21800,4.59,2156624900,796,G.I. Generation,Albania,1987,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,274300,3.28,2156624900,796,Boomers,Albania,1987,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [450]:
X = df_age_sex_gen_coun_OHE.drop(['generation', 'suicides/100k_pop', 'country', 'year', 'suicides_no', 'population'], axis=1)
y = df_age_sex_gen_coun_OHE['suicides/100k_pop']

In [451]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [452]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

estimator = LinearRegression()
estimator.fit(X_train, y_train) # Обучение


y_pred = estimator.predict(X_test)

print("R2: \t", r2_score(y_test, y_pred))
print("RMSE: \t", np.sqrt(mean_squared_error(y_test, y_pred)))
print("MAE: \t", mean_absolute_error(y_test, y_pred))

R2: 	 0.4990121073681648
RMSE: 	 12.695160599624229
MAE: 	 8.270679447975112


**Вывод:** число самоубийств на душу населения сильно зависит от страны проживания, так как ее включение позволило повысить достоверность модели. При этом в модели обучена без 'suicides_no', 'population'. Если их включить, то значение коэффициента R2 поднимится всего до 0,53